In [17]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import pandas as pd
from tqdm import tqdm  # Per la barra di caricamento
import platform
from torchvision.models import AlexNet_Weights, GoogLeNet_Weights
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import csv


In [18]:
def test_model(model_path, type, average, num_classes=38):
    # Percorso al CSV (puoi definirlo qui)
    csv_path = 'final_dataset/test/df_paths_test.csv'

    # Funzione per caricare il modello
    def load_model(model_path, device):
        if type == 'googlenet':
            model = models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
            model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modifica il classificatore finale
            model.aux1 = None  # Disabilita le teste ausiliarie
            model.aux2 = None
        elif type == 'alexnet':
            model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)  # Modifica il classificatore finale
        elif type == 'resnet50':
            model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes)
        
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        return model

    # Funzione per caricare un'immagine e applicare le trasformazioni
    def process_image(image_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)
        return image

    def calculate_metrics(labels, outputs):
        # Convert labels to binary
        # labels_binary = [1 if label == 'Target' else 0 for label in labels]
        # # Convert outputs to binary
        # outputs_binary = [1 if output == 'Target' else 0 for output in outputs]
        # 
        
        accuracy = accuracy_score(labels, outputs)
        precision = precision_score(labels, outputs, average=average)
        recall = recall_score(labels, outputs, average=average, zero_division=1)
        f1 = f1_score(labels, outputs, average=average)
        return accuracy, precision, recall, f1

    def save_metrics(accuracy, precision, recall, f1, path):
        with open(path, 'w', newline='') as csvfile:
            fieldnames = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow({
                'test_accuracy': test_accuracy,
                'test_precision': test_precision,
                'test_recall': test_recall,
                'test_f1': test_f1
            })

    # Carica il dispositivo (GPU se disponibile, altrimenti CPU)
    if platform.system() == 'Windows':
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    # Carica il modello
    model = load_model(model_path, device)

    # Leggi il CSV
    df = pd.read_csv(csv_path)

    # Inizializza le liste per le predizioni e i target
    predictions = []
    targets = []

    # Itera attraverso i dati di test e fai previsioni
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Testing"):
        image_path = row['FilePath']  # Assicurati che il CSV abbia una colonna 'FilePath'
        image = process_image(image_path).to(device)

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
        
        predictions.append(predicted.item())
        targets.append(row['Classe'])  # Assicurati che il CSV abbia una colonna 'Classe' con le etichette delle classi

    # Converti le etichette delle classi a numeri se necessario
    unique_labels = df['Classe'].unique()
    label_to_index = {label: index for index, label in enumerate(unique_labels)}
    targets = [label_to_index[label] for label in targets]
    
    test_accuracy, test_precision, test_recall, test_f1 = calculate_metrics(predictions, targets)
    
    name = model_path.split('/')[1]
    save_metrics(test_accuracy, test_precision, test_recall, test_f1, f'test_results/{name}.csv')

    # Stampa i risultati
    print(f'Accuracy: {test_accuracy:.4f} - Precision: {test_precision:.4f} - Recall: {test_recall:.4f} - F1: {test_f1:.4f}')

# Test del modello con dataset bilanciato

In [19]:
test_model('models/test11_alex_net_one_hot/best_model.pt', 'alexnet', 'weighted')

Testing: 100%|██████████| 7689/7689 [02:15<00:00, 56.60it/s]

Accuracy: 0.1275 - Precision: 0.1376 - Recall: 0.1275 - F1: 0.1297


# Test con dataset sbilanciato

## GoogLeNet

In [20]:
test_model('models/test17_google_net_one_hot_unbalanced/best_model.pt', 'googlenet', 'micro')

Testing: 100%|██████████| 7689/7689 [04:03<00:00, 31.53it/s]

Accuracy: 0.1966 - Precision: 0.1966 - Recall: 0.1966 - F1: 0.1966


In [29]:
test_model('models/test18_google_net_one_hot_unbalanced/best_model.pt', 'googlenet', 'weighted')

Testing: 100%|██████████| 7689/7689 [03:45<00:00, 34.16it/s]

Accuracy: 0.2246 - Precision: 0.2326 - Recall: 0.2246 - F1: 0.2262


## AlexNet

In [22]:
test_model('models/test15_alex_net_one_hot_unbalanced/best_model.pt', 'alexnet', 'weighted')

Testing: 100%|██████████| 7689/7689 [02:14<00:00, 57.27it/s]

Accuracy: 0.2367 - Precision: 0.2617 - Recall: 0.2367 - F1: 0.2419


In [25]:
test_model('models/test16_alex_net_one_hot_unbalanced/best_model.pt', 'alexnet', 'micro')

Testing: 100%|██████████| 7689/7689 [02:11<00:00, 58.56it/s]

Accuracy: 0.2424 - Precision: 0.2424 - Recall: 0.2424 - F1: 0.2424


## ResNet50

In [27]:
test_model('models/test13_res_net_one_hot_unbalanced/best_model.pt', 'resnet50', 'weighted')

Testing: 100%|██████████| 7689/7689 [03:46<00:00, 34.00it/s]

Accuracy: 0.1630 - Precision: 0.2651 - Recall: 0.1630 - F1: 0.1883


In [28]:
test_model('models/test14_res_net_one_hot_unbalanced/best_model.pt', 'resnet50', 'micro')

Testing: 100%|██████████| 7689/7689 [03:41<00:00, 34.66it/s]

Accuracy: 0.1857 - Precision: 0.1857 - Recall: 0.1857 - F1: 0.1857
